<a href="https://colab.research.google.com/github/aTunass/Crawling_text_img/blob/main/crawl_news(text).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Import necessary libraries**

Lưu ý: khi dùng trên máy local, cần check phiên bản của chrome và chronium, đối với colab các phiên bản ubuntu 20.04 không có sự hỗ trợ của chronium, khi cài driver này từ bên ngoài sẽ chỉ dùng được khi có options headless

In [2]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.Gw2vmjnStM/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.sgEwVoEwyz/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.ka74kT7wFq/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [3]:
import pandas as pd
import os
import requests
import time
import random

from io import BytesIO
from PIL import Image
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Initialize google chrome browser
service = Service(executable_path="chromedriver")
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)
# url = 'https://www.python.org/'
# driver.get(url)
# print(driver.title)

## **2. Crawl**

In [5]:


# Create a folder for storing articles
root_dir = './vn_news_corpus'
os.makedirs(root_dir, exist_ok=True)
n_pages = 1 # Change if you want more articles
article_id = 0

for page_idx in tqdm(range(n_pages)):
    # Access to table page
    main_url = f'https://vietnamnet.vn/thoi-su-page{page_idx}'
    driver.get(main_url)

    # Get list of articles (list of URLs)
    news_lst_xpath = '//div[@class="topStory-15nd"]/div/div[1]/a'
    news_tags = driver.find_elements(
        By.XPATH,
        news_lst_xpath
    )
    news_page_urls = [
        news_tag.get_attribute('href') \
            for news_tag in news_tags
    ]

    for news_page_url in news_page_urls:
        # Access to article page
        driver.get(news_page_url)
        time.sleep(1)

        # Try to get main content tag
        main_content_xpath = '//div[@class="content-detail content-mobile-change"]'
        try:
            main_content_tag = driver.find_element(
                By.XPATH,
                main_content_xpath
            )
        except:
          main_content_xpath = '//div[@class="class="content-detail image-full-width content-mobile-change""]'
          try:
              main_content_tag = driver.find_element(
                  By.XPATH,
                  main_content_xpath
              )
          except:
            continue
        # Ignore video article
        video_content_xpath = '//div[@class="video-detail"]'
        try:
            video_content_tag = main_content_tag.find_element(
                By.XPATH,
                video_content_xpath
            )
            continue
        except:
            pass
        # Get title (h1 tag)
        try:
          title = main_content_tag.find_element(
              By.TAG_NAME,
              'h1'
          ).text.strip()
        except:
            title = ''
        # Get abstract (h2 tag)
        try:
          abstract = main_content_tag.find_element(
              By.TAG_NAME,
              'h2'
          ).text.strip()
        except:
            abstract = ''
        # Get author name (span tag)
        try:
            author_xpath = '//span[@class="article-detail-author-wrapper "]'
            author = main_content_tag.find_element(
                By.XPATH,
                author_xpath
            ).text.strip()
        except:
            author = ''
        # Get paragraphs (all p tags in div "maincontent main-content")
        paragraphs_xpath = '//div[@class="maincontent main-content"]/p'
        paragraphs_tags = main_content_tag.find_elements(
            By.XPATH,
            paragraphs_xpath
        )
        paragraphs_lst = [
            paragraphs_tag.text.strip() \
                for paragraphs_tag in paragraphs_tags
        ]
        paragraphs = ' '.join(paragraphs_lst)
        # Combine title, abstract, authoor and paragraphs
        final_content_lst = [title, abstract, paragraphs, author]
        final_content = '\n\n'.join(final_content_lst)

        # Save artile to .txt file
        article_filename = f'article_{article_id:05d}.txt'
        article_savepath = os.path.join(
            root_dir,
            article_filename
        )
        article_id += 1
        with open(article_savepath, 'w') as f:
            f.write(final_content)

        # Move back to previous page
        driver.back()

100%|██████████| 2/2 [05:58<00:00, 179.41s/it]


In [6]:
!zip -r vn_news_corpus.zip vn_news_corpus

  adding: vn_news_corpus/ (stored 0%)
  adding: vn_news_corpus/article_00015.txt (deflated 65%)
  adding: vn_news_corpus/article_00021.txt (deflated 61%)
  adding: vn_news_corpus/article_00001.txt (deflated 31%)
  adding: vn_news_corpus/article_00017.txt (deflated 64%)
  adding: vn_news_corpus/article_00026.txt (deflated 55%)
  adding: vn_news_corpus/article_00002.txt (deflated 58%)
  adding: vn_news_corpus/article_00006.txt (deflated 46%)
  adding: vn_news_corpus/article_00003.txt (deflated 56%)
  adding: vn_news_corpus/article_00025.txt (deflated 56%)
  adding: vn_news_corpus/article_00012.txt (deflated 62%)
  adding: vn_news_corpus/article_00024.txt (deflated 60%)
  adding: vn_news_corpus/article_00000.txt (deflated 58%)
  adding: vn_news_corpus/article_00009.txt (deflated 61%)
  adding: vn_news_corpus/article_00013.txt (deflated 59%)
  adding: vn_news_corpus/article_00023.txt (deflated 54%)
  adding: vn_news_corpus/article_00020.txt (deflated 59%)
  adding: vn_news_corpus/article_0

In [7]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
!cp '/content/vn_news_corpus.zip' '/content/gdrive/MyDrive'

In [1]:
!rm -r /content/vn_news_corpus
